## **Обучени модели классификации заказов**

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import Pipeline
import joblib

Тестирование результатов классификации заказов будет проводится на ещё не выполненных заказах, для симуляции некого инференса

In [2]:
df = pd.read_csv('../data/cases_labeled.csv', sep=';')
# Разделим данные на обучающую и тестовую выборки
X_train = df.loc[df['Выполнено'] == 'да']['Описание']
X_test = df.loc[df['Выполнено'] == 'нет']['Описание']
y_train = df.loc[df['Выполнено'] == 'да']['Тип поручения']
y_test = df.loc[df['Выполнено'] == 'нет']['Тип поручения']

In [3]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=5000, stop_words=['russian'])),  # Векторизация текста
    ('clf', LogisticRegression(max_iter=1000, random_state=42))  # Модель классификации
])

In [4]:
# Обучаем модель
pipeline.fit(X_train, y_train)

# Прогнозируем на тестовых данных
y_pred = pipeline.predict(X_test)

# Оценка качества модели
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=['Чудовище', 'Спасение', 'Поиск', 'Монстры/разбойники']))

Accuracy: 1.0
Classification Report:
                    precision    recall  f1-score   support

          Чудовище       1.00      1.00      1.00         3
          Спасение       1.00      1.00      1.00         5
             Поиск       1.00      1.00      1.00         8
Монстры/разбойники       1.00      1.00      1.00         3

          accuracy                           1.00        19
         macro avg       1.00      1.00      1.00        19
      weighted avg       1.00      1.00      1.00        19



Для оценки качества модели на "новых" заказах, мной были сгенирированы новые текста (немного отличающиеся от исходных) с метками

In [5]:
val = pd.read_csv('../data/test.csv', sep=';')
val['Тип поручения'] = val['Тип поручения'].astype(int)
X_val = val['Описание']
y_val = val['Тип поручения']

y_pred = pipeline.predict(X_val)

# Оценка качества модели
print("Accuracy:", accuracy_score(y_val, y_pred))
print("Classification Report:")
print(classification_report(y_val, y_pred, target_names=['Чудовище', 'Спасение', 'Поиск', 'Монстры/разбойники']))

Accuracy: 0.8947368421052632
Classification Report:
                    precision    recall  f1-score   support

          Чудовище       1.00      1.00      1.00         5
          Спасение       1.00      0.50      0.67         4
             Поиск       1.00      1.00      1.00         4
Монстры/разбойники       0.75      1.00      0.86         6

          accuracy                           0.89        19
         macro avg       0.94      0.88      0.88        19
      weighted avg       0.92      0.89      0.88        19



In [6]:
# Сохранение пайплайна в файл
joblib.dump(pipeline, '../models/classification_model.pkl')

['../models/classification_model.pkl']